In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Import tools
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Install PandaSql and Stats Models
!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandasql as ps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=f82d6f15c5f64e94f453de0b47e5185a6bbe8fd4c57fa68026b6f60bc7ad5fc3
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [ ]:
#read compustat file

file = "/content/gdrive/MyDrive/FinancePaperFIMA/Compustat2018Through2022.csv"
df=pd.read_csv(file)
df.head(10)
Compustat2018Through2022=df
print(Compustat2018Through2022.columns)
print(Compustat2018Through2022)

file= "/content/gdrive/MyDrive/FinancePaperFIMA/CRSP2020Through2022.csv"
df=pd.read_csv(file)
df.head(10)
CRSP2020Through2022=df
print(CRSP2020Through2022.columns)
print(Compustat2018Through2022)


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,18,26,30,647,669,674) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'fyr', 'indfmt', 'consol',
       'popsrc', 'datafmt', 'tic',
       ...
       'priusa', 'sic', 'spcindcd', 'spcseccd', 'spcsrc', 'state', 'stko',
       'weburl', 'dldte', 'ipodate'],
      dtype='object', length=680)
         gvkey  datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
0         1004  20180228    2017   3.0    5   INDL      C      D     STD   
1         1004  20180531    2017   4.0    5   INDL      C      D     STD   
2         1004  20180831    2018   1.0    5   INDL      C      D     STD   
3         1004  20181130    2018   2.0    5   INDL      C      D     STD   
4         1004  20190228    2018   3.0    5   INDL      C      D     STD   
...        ...       ...     ...   ...  ...    ...    ...    ...     ...   
190780  351590  20201231    2020   4.0   12   INDL      C      D     STD   
190781  351590  20210331    2021   1.0   12   INDL      C      D     STD   
190782  351590  20210630    2021   2.0   12   INDL

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,9,18,49,57) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP',
       'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT',
       'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT',
       'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG',
       'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM',
       'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD',
       'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD',
       'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')
         gvkey  datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
0         1004  20180228    2017   3.0    5   INDL      C      D     STD   
1         1004  20180531    2017   4.0    5   INDL      C      D     STD   
2         1004  20180831    2018   1.0    5   INDL      C      D     STD   
3 

In [ ]:
### CLEAN COMPUSTAT DATA ###
Compustat2018Through2022["datadate"]=pd.to_datetime(Compustat2018Through2022["datadate"],format="%Y%m%d")
Compustat2018Through2022["rdq"]=pd.to_datetime(Compustat2018Through2022["rdq"],format="%Y%m%d")
Compustat2018Through2022["cshoq"]=pd.to_numeric(Compustat2018Through2022["cshoq"], errors='coerce')
Compustat2018Through2022["ceqq"]=pd.to_numeric(Compustat2018Through2022["ceqq"], errors='coerce')
Compustat2018Through2022["ibq"]=pd.to_numeric(Compustat2018Through2022["ibq"], errors='coerce')
Compustat2018Through2022["saleq"]=pd.to_numeric(Compustat2018Through2022["saleq"], errors='coerce')
Compustat2018Through2022["prccq"]=pd.to_numeric(Compustat2018Through2022["prccq"], errors='coerce')

Compustat2018Through2022['indfmt']=Compustat2018Through2022['indfmt'].apply(str)
Compustat2018Through2022['datafmt']=Compustat2018Through2022['datafmt'].apply(str)

Compustat2018Through2022 = Compustat2018Through2022[Compustat2018Through2022['indfmt']=="INDL"]
Compustat2018Through2022 = Compustat2018Through2022[Compustat2018Through2022['datafmt']=="STD"]



In [ ]:
from IPython.display import HTML, display

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("darkgrid")

from tabulate import tabulate

In [ ]:
#####ONLY TAKE DATES AFTER 2020#####
from sqlite3 import connect
conn = connect(':memory:')

Compustat2018Through2022.to_sql("Compustat2018Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT *
        FROM Compustat2018Through2022
        WHERE Compustat2018Through2022.datadate>="20200101"
        '''
Compustat2020Through2022 = pd.read_sql(query,conn)

compustat2020Through2022=Compustat2020Through2022
crsp2020Through2022=CRSP2020Through2022

In [ ]:
print(crsp2020Through2022)

         PERMNO      date  NAMEENDT  SHRCD  EXCHCD SICCD    NCUSIP TICKER  \
0         10026  20200102       NaN   11.0     3.0  2052  46603210   JJSF   
1         10026  20200103       NaN   11.0     3.0  2052  46603210   JJSF   
2         10026  20200106       NaN   11.0     3.0  2052  46603210   JJSF   
3         10026  20200107       NaN   11.0     3.0  2052  46603210   JJSF   
4         10026  20200108       NaN   11.0     3.0  2052  46603210   JJSF   
...         ...       ...       ...    ...     ...   ...       ...    ...   
4715937   93436  20220325       NaN   11.0     3.0  9999  88160R10   TSLA   
4715938   93436  20220328       NaN   11.0     3.0  9999  88160R10   TSLA   
4715939   93436  20220329       NaN   11.0     3.0  9999  88160R10   TSLA   
4715940   93436  20220330       NaN   11.0     3.0  9999  88160R10   TSLA   
4715941   93436  20220331       NaN   11.0     3.0  9999  88160R10   TSLA   

                         COMNAM SHRCLS  ... CFACPR  CFACSHR     OPENPRC  \


In [ ]:
### CLEAN CRSP DATA ###
crsp2020Through2022["date"]=pd.to_datetime(crsp2020Through2022["date"],format="%Y%m%d")

#List of tickers for COMPUstat

compustat2020Through2022.to_sql("compustat2020Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT compustat2020Through2022.tic, compustat2020Through2022.rdq
        FROM compustat2020Through2022
        WHERE compustat2020Through2022.tic<>"" and compustat2020Through2022.rdq<>""
        '''
compustat2020Through2022uniquegvkeysandrdqs = pd.read_sql(query,conn)
print(compustat2020Through2022uniquegvkeysandrdqs)

          tic                  rdq
0         AIR  2018-03-20 00:00:00
1         AIR  2018-07-10 00:00:00
2         AIR  2018-09-25 00:00:00
3         AIR  2018-12-18 00:00:00
4         AIR  2019-03-19 00:00:00
...       ...                  ...
122483   NXTP  2021-10-20 00:00:00
122484   NXTP  2022-01-13 00:00:00
122485  IVCGF  2021-10-29 00:00:00
122486  DTRUY  2021-11-26 00:00:00
122487  DTRUY  2022-03-24 00:00:00

[122488 rows x 2 columns]


In [ ]:
### GET THE CRSP TICKERS AND DATES ###
crsp2020Through2022.to_sql("crsp2020Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT crsp2020Through2022.TICKER, crsp2020Through2022.date
        FROM crsp2020Through2022
        '''
crsp2020Through2022uniqueticsanddates = pd.read_sql(query, conn)
print(crsp2020Through2022uniqueticsanddates)

        TICKER                 date
0         JJSF  2020-01-02 00:00:00
1         JJSF  2020-01-03 00:00:00
2         JJSF  2020-01-06 00:00:00
3         JJSF  2020-01-07 00:00:00
4         JJSF  2020-01-08 00:00:00
...        ...                  ...
4692497   TSLA  2022-03-25 00:00:00
4692498   TSLA  2022-03-28 00:00:00
4692499   TSLA  2022-03-29 00:00:00
4692500   TSLA  2022-03-30 00:00:00
4692501   TSLA  2022-03-31 00:00:00

[4692502 rows x 2 columns]


In [ ]:
###FULL LIST OF CRSP TICKERS AND DATES AND MAKE THEM THE LEFT TABLE AND THEN LEFT JOIN EARNINGS ANOUNCEMENT DATES AS RDQ###
crsp2020Through2022uniqueticsanddates.to_sql("crsp2020Through2022uniqueticsanddates", conn, if_exists='replace')
compustat2020Through2022uniquegvkeysandrdqs.to_sql("compustat2020Through2022uniquegvkeysandrdqs", conn, if_exists='replace')
query = '''SELECT DISTINCT crsp2020Through2022uniqueticsanddates.TICKER, crsp2020Through2022uniqueticsanddates.date, compustat2020Through2022uniquegvkeysandrdqs.rdq
        FROM crsp2020Through2022uniqueticsanddates
        LEFT JOIN compustat2020Through2022uniquegvkeysandrdqs
        ON crsp2020Through2022uniqueticsanddates.TICKER = compustat2020Through2022uniquegvkeysandrdqs.tic
        and crsp2020Through2022uniqueticsanddates.date=compustat2020Through2022uniquegvkeysandrdqs.rdq
        ORDER BY crsp2020Through2022uniqueticsanddates.TICKER, crsp2020Through2022uniqueticsanddates.date
        '''
crspcompustat2020Through2022uniqueticsanddatesandrdq = pd.read_sql(query,conn)
print(crspcompustat2020Through2022uniqueticsanddatesandrdq)

        TICKER                 date   rdq
0         None  2020-01-02 00:00:00  None
1         None  2020-01-03 00:00:00  None
2         None  2020-01-06 00:00:00  None
3         None  2020-01-07 00:00:00  None
4         None  2020-01-08 00:00:00  None
...        ...                  ...   ...
4692497   ZYXI  2022-03-25 00:00:00  None
4692498   ZYXI  2022-03-28 00:00:00  None
4692499   ZYXI  2022-03-29 00:00:00  None
4692500   ZYXI  2022-03-30 00:00:00  None
4692501   ZYXI  2022-03-31 00:00:00  None

[4692502 rows x 3 columns]


In [ ]:
###DATES THAT ARE THREE DAYS AHEAD THE EARNINGS ANNOUNCEMENT DATE###
#sort the table dailyret1004
crspcompustat2020Through2022uniqueticsanddatesandrdq.sort_values(by=['TICKER','date'], ascending=False)
print(crspcompustat2020Through2022uniqueticsanddatesandrdq.head(10))

#Calculating the 3 day return window
crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus1'] = crspcompustat2020Through2022uniqueticsanddatesandrdq.groupby(['TICKER'])['rdq'].shift(1)
crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus2'] = crspcompustat2020Through2022uniqueticsanddatesandrdq.groupby(['TICKER'])['rdq'].shift(2)
crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus3'] = crspcompustat2020Through2022uniqueticsanddatesandrdq.groupby(['TICKER'])['rdq'].shift(3)
crspcompustat2020Through2022uniqueticsanddatesandrdq['three_days_after_earnings_date_ifearningsduringday'] = crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus2']
crspcompustat2020Through2022uniqueticsanddatesandrdq['three_days_after_earnings_date_ifearningsafterhours'] = crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus3']


crspcompustat2020Through2022uniqueticsanddatesandrdq.to_sql("crspcompustat2020Through2022uniqueticsanddatesandrdq", conn, if_exists='replace')
query = '''SELECT DISTINCT crspcompustat2020Through2022uniqueticsanddatesandrdq.TICKER, crspcompustat2020Through2022uniqueticsanddatesandrdq.date,
        crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq, crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus1,
        crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus2, crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus3
        FROM crspcompustat2020Through2022uniqueticsanddatesandrdq
        WHERE crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq<>"NaN" or crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus1<>"NaN"
        or crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus2<>"Nan" or crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus3<>"NaN"
        '''

crspcompustat2020Through2022threedaysfollowingrdq = pd.read_sql(query, conn)
print(crspcompustat2020Through2022threedaysfollowingrdq)

  TICKER                 date   rdq
0   None  2020-01-02 00:00:00  None
1   None  2020-01-03 00:00:00  None
2   None  2020-01-06 00:00:00  None
3   None  2020-01-07 00:00:00  None
4   None  2020-01-08 00:00:00  None
5   None  2020-01-09 00:00:00  None
6   None  2020-01-10 00:00:00  None
7   None  2020-01-13 00:00:00  None
8   None  2020-01-14 00:00:00  None
9   None  2020-01-15 00:00:00  None
       TICKER                 date                  rdq            rdq_plus1  \
0           A  2020-02-18 00:00:00  2020-02-18 00:00:00                 None   
1           A  2020-02-19 00:00:00                 None  2020-02-18 00:00:00   
2           A  2020-02-20 00:00:00                 None                 None   
3           A  2020-02-21 00:00:00                 None                 None   
4           A  2020-05-21 00:00:00  2020-05-21 00:00:00                 None   
...       ...                  ...                  ...                  ...   
172429   ZYXI  2021-11-05 00:00:00          

In [ ]:
###WE NEXT ADD THE RETURN DATA TO THE DATES AND TICKER TABLE FROM ABOVE###
crspcompustat2020Through2022threedaysfollowingrdq.to_sql("crspcompustat2020Through2022threedaysfollowingrdq", conn, if_exists='replace')
crsp2020Through2022.to_sql("crsp2020Through2022", conn, if_exists='replace')
query = ''' SELECT DISTINCT crspcompustat2020Through2022threedaysfollowingrdq.TICKER, crspcompustat2020Through2022threedaysfollowingrdq.date,
        crspcompustat2020Through2022threedaysfollowingrdq.rdq, crsp2020Through2022.RET
        FROM crspcompustat2020Through2022threedaysfollowingrdq
        LEFT JOIN crsp2020Through2022
        ON crspcompustat2020Through2022threedaysfollowingrdq.TICKER=crsp2020Through2022.TICKER and
        crspcompustat2020Through2022threedaysfollowingrdq.date=crsp2020Through2022.date
        '''
crspcompustat2020Through2022threedaysfollowingrdqwithreturns = pd.read_sql(query,conn)
print(crspcompustat2020Through2022threedaysfollowingrdqwithreturns)

       TICKER                 date                  rdq        RET
0           A  2020-02-18 00:00:00  2020-02-18 00:00:00  -0.012002
1           A  2020-02-19 00:00:00                 None   0.006605
2           A  2020-02-20 00:00:00                 None  -0.011834
3           A  2020-02-21 00:00:00                 None   0.008655
4           A  2020-05-21 00:00:00  2020-05-21 00:00:00  -0.011749
...       ...                  ...                  ...        ...
172904   ZYXI  2021-11-05 00:00:00                 None  -0.009852
172905   ZYXI  2022-02-24 00:00:00  2022-02-24 00:00:00  -0.012012
172906   ZYXI  2022-02-25 00:00:00                 None  -0.004559
172907   ZYXI  2022-02-28 00:00:00                 None  -0.039695
172908   ZYXI  2022-03-01 00:00:00                 None  -0.052464

[172909 rows x 4 columns]


In [ ]:
### WE NEXT MATCH THE DATES TO COMPUSTAT DATADATE ###
crspcompustat2020Through2022threedaysfollowingrdqwithreturns.to_sql("crspcompustat2020Through2022threedaysfollowingrdqwithreturns", conn, if_exists='replace')
compustat2020Through2022.to_sql("compustat2020Through2022", conn, if_exists='replace')
query ='''SELECT DISTINCT crspcompustat2020Through2022threedaysfollowingrdqwithreturns.Ticker, crspcompustat2020Through2022threedaysfollowingrdqwithreturns.date,
       crspcompustat2020Through2022threedaysfollowingrdqwithreturns.rdq, crspcompustat2020Through2022threedaysfollowingrdqwithreturns.RET,
       compustat2020Through2022.datadate, compustat2020Through2022.dvy, compustat2020Through2022.saleq, compustat2020Through2022.ibq,
       compustat2020Through2022.fyearq, compustat2020Through2022.fqtr, compustat2020Through2022.fyr, compustat2020Through2022.ceqq, compustat2020Through2022.cshoq,
       compustat2020Through2022.prccq
       FROM crspcompustat2020Through2022threedaysfollowingrdqwithreturns
       LEFT JOIN compustat2020Through2022
       ON crspcompustat2020Through2022threedaysfollowingrdqwithreturns.TICKER=compustat2020Through2022.tic and
       crspcompustat2020Through2022threedaysfollowingrdqwithreturns.rdq=compustat2020Through2022.rdq
       '''
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsanddatadate = pd.read_sql(query,conn)
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr=crspcompustat2020Through2022threedaysfollowingrdqwithreturnsanddatadate
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsanddatadate, headers='keys', tablefmt='presto'))

In [ ]:
### CALCULATE THE THREE DAY RETURN FOLLOWING THE EARNINGS ANNOUNCEMENT ###
table=crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr
table['ret_1'] = table.groupby(['TICKER'])['RET'].shift(-1)
table['ret_2'] = table.groupby(['TICKER'])['RET'].shift(-2)
table['ret_3'] = table.groupby(['TICKER'])['RET'].shift(-3)
table['RET']=pd.to_numeric(table['RET'],errors='coerce')
table['ret_1']=pd.to_numeric(table['ret_1'],errors='coerce')
table['ret_2']=pd.to_numeric(table['ret_2'],errors='coerce')
table['ret_3']=pd.to_numeric(table['ret_3'],errors='coerce')
table['three_day_return_ifearningsduringday'] = (1+table['RET'])*(1+table['ret_1'])*(1+table['ret_2'])-1
table['three_day_return_ifearningsafterhours'] = (1+table['ret_1'])*(1+table['ret_2'])*(1+table['ret_3'])-1
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr=table
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr, headers='keys', tablefmt='presto'))

In [ ]:
print(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr)

       TICKER                 date                  rdq       RET  \
0           A  2020-02-18 00:00:00  2020-02-18 00:00:00 -0.012002   
1           A  2020-02-19 00:00:00                 None  0.006605   
2           A  2020-02-20 00:00:00                 None -0.011834   
3           A  2020-02-21 00:00:00                 None  0.008655   
4           A  2020-05-21 00:00:00  2020-05-21 00:00:00 -0.011749   
...       ...                  ...                  ...       ...   
173358   ZYXI  2021-11-05 00:00:00                 None -0.009852   
173359   ZYXI  2022-02-24 00:00:00  2022-02-24 00:00:00 -0.012012   
173360   ZYXI  2022-02-25 00:00:00                 None -0.004559   
173361   ZYXI  2022-02-28 00:00:00                 None -0.039695   
173362   ZYXI  2022-03-01 00:00:00                 None -0.052464   

                   datadate      dvy     saleq      ibq  fyearq  fqtr   fyr  \
0       2020-01-31 00:00:00   56.000  1357.000  197.000  2020.0   1.0  10.0   
1            

In [ ]:
### ONLY KEEP ROWS WITH AN EARNINGS ANNOUNCEMENT ###
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.to_sql("crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr", conn, if_exists='replace')
query = '''SELECT DISTINCT crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.TICKER,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.date,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.rdq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.RET,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.datadate,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.dvy,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.saleq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ibq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.fyearq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.fqtr,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.fyr,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ceqq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.cshoq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.prccq,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ret_1,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ret_2,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ret_3,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.three_day_return_ifearningsduringday,
        crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.three_day_return_ifearningsafterhours
        FROM crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr
        WHERE crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.rdq<>""
        '''
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query,conn)
print(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq)

      TICKER                 date                  rdq       RET  \
0          A  2020-02-18 00:00:00  2020-02-18 00:00:00 -0.012002   
1          A  2020-05-21 00:00:00  2020-05-21 00:00:00 -0.011749   
2          A  2020-08-18 00:00:00  2020-08-18 00:00:00  0.003589   
3          A  2020-11-23 00:00:00  2020-11-23 00:00:00  0.011904   
4          A  2021-02-16 00:00:00  2021-02-16 00:00:00 -0.000234   
...      ...                  ...                  ...       ...   
43886   ZYXI  2021-02-25 00:00:00  2021-02-25 00:00:00 -0.054289   
43887   ZYXI  2021-04-29 00:00:00  2021-04-29 00:00:00 -0.014145   
43888   ZYXI  2021-07-29 00:00:00  2021-07-29 00:00:00 -0.027875   
43889   ZYXI  2021-11-02 00:00:00  2021-11-02 00:00:00 -0.004766   
43890   ZYXI  2022-02-24 00:00:00  2022-02-24 00:00:00 -0.012012   

                  datadate      dvy     saleq      ibq  fyearq  fqtr   fyr  \
0      2020-01-31 00:00:00   56.000  1357.000  197.000  2020.0   1.0  10.0   
1      2020-04-30 00:00:00 

In [ ]:
### 2019 ROWS FROM COMPUSTAT ###
Compustat2018Through2022.to_sql("Compustat2018Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT tic as TICKER, rdq, datadate, dvy, saleq, ibq, fyearq, fqtr, fyr, ceqq, cshoq, prccq
        FROM Compustat2018Through2022
        WHERE Compustat2018Through2022.fyearq == "2019"
        ORDER BY TICKER, datadate
        '''
compustat2019ticrdqdatadatesaleqibqfyearqfqtrfyrceqqcshoqprccq = pd.read_sql(query,conn)
#print(tabulate(compustat2019ticrdqdatadatesaleqibqfyearqfqtrfyrceqqcshoqprccq, headers='keys', tablefmt='presto'))
## change it back to and Compustat2018through2022.tic<>""

In [ ]:
### CONCATENATE THE 2019 TABLE and the 2020-2022 TABLE ###
Table2019to2022 = crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019 = compustat2019ticrdqdatadatesaleqibqfyearqfqtrfyrceqqcshoqprccq
Table2020to2022 = crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019to2022 = Table2019.append(Table2020to2022)
print(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.head(10))

Table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = Table2019to2022
#print(tabulate(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq, headers = 'keys', tablefmt = 'presto'))


  TICKER                 date                  rdq       RET  \
0      A  2020-02-18 00:00:00  2020-02-18 00:00:00 -0.012002   
1      A  2020-05-21 00:00:00  2020-05-21 00:00:00 -0.011749   
2      A  2020-08-18 00:00:00  2020-08-18 00:00:00  0.003589   
3      A  2020-11-23 00:00:00  2020-11-23 00:00:00  0.011904   
4      A  2021-02-16 00:00:00  2021-02-16 00:00:00 -0.000234   
5      A  2021-05-25 00:00:00  2021-05-25 00:00:00 -0.000825   
6      A  2021-08-17 00:00:00  2021-08-17 00:00:00 -0.007157   
7      A  2021-11-22 00:00:00  2021-11-22 00:00:00 -0.009251   
8      A  2022-02-22 00:00:00  2022-02-22 00:00:00 -0.011814   
9     AA  2020-01-15 00:00:00  2020-01-15 00:00:00 -0.009327   

              datadate    dvy   saleq    ibq  fyearq  fqtr   fyr    ceqq  \
0  2020-01-31 00:00:00   56.0  1357.0  197.0  2020.0   1.0  10.0  4848.0   
1  2020-04-30 00:00:00  111.0  1238.0  101.0  2020.0   2.0  10.0  4768.0   
2  2020-07-31 00:00:00  167.0  1261.0  199.0  2020.0   3.0  10.0  4

In [ ]:
#CALCULATE MARKET OF EQUITY AND BOOK TO MARKET#
print(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.columns)
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, rdq, datadate, dvy, saleq, ibq, fyearq, fqtr,
        fyr, ceqq, cshoq, prccq, date, RET, ret_1, ret_2,
        ret_3, three_day_return_ifearningsduringday, three_day_return_ifearningsafterhours, cshoq*prccq as market_value_of_equity,
        ceqq / (cshoq*prccq) as book_to_market_timet
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query, conn)
#print(tabulate(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq, headers='keys', tablefmt='presto'))

#CALCULATE BOOK TO MARKET QUARTERLY THIRDS#
#If the book to market is negative, then set it equal to 0.5
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.loc[crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet < 0, 'book_to_market_timet'] = 0.5
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, book_to_market_timet
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE book_to_market_timet<>""
        '''
booktomarketdata2019Through2022 = pd.read_sql(query,conn)
booktomarketdata2019Through2022['percentile'] = booktomarketdata2019Through2022.groupby(['datadate'])['book_to_market_timet'].rank(pct=True)
booktomarketdata2019Through2022.loc[booktomarketdata2019Through2022['percentile'] <= .33, 'FirstThird'] = True
booktomarketdata2019Through2022.loc[booktomarketdata2019Through2022['percentile'] >= .67, 'LastThird'] = True
#print(tabulate(booktomarketdata2019Through2022, headers='keys', tablefmt='presto'))

# CALCULATE DIVIDEND QUARTERLY QUARTILES #

crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, dvy/market_value_of_equity as dividendyield
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE market_value_of_equity<>""
        '''
dividendyielddata2019Through2022 = pd.read_sql(query,conn)
dividendyielddata2019Through2022['percentile'] = dividendyielddata2019Through2022.groupby(['datadate'])['dividendyield'].rank(pct=True)
dividendyielddata2019Through2022.loc[dividendyielddata2019Through2022['percentile'] <= .25, 'FirstQuartile'] = True
dividendyielddata2019Through2022.loc[dividendyielddata2019Through2022['percentile'] >= .75, 'FourthQuartile'] = True
#print(tabulate(dividendyielddata2019Through2022, headers='keys', tablefmt='presto'))

Index(['TICKER', 'rdq', 'datadate', 'dvy', 'saleq', 'ibq', 'fyearq', 'fqtr',
       'fyr', 'ceqq', 'cshoq', 'prccq', 'date', 'RET', 'ret_1', 'ret_2',
       'ret_3', 'three_day_return_ifearningsduringday',
       'three_day_return_ifearningsafterhours'],
      dtype='object')


In [ ]:
# MERGE THE BOOK TO MARKET AND DIVIDEND DATA BACK IN #

crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
booktomarketdata2019Through2022.to_sql("booktomarketdata2019Through2022", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
        booktomarketdata2019Through2022.FirstThird as booktomarketfirstthird, booktomarketdata2019Through2022.LastThird as booktomarketlastthird
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        LEFT JOIN booktomarketdata2019Through2022
        ON crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER=booktomarketdata2019Through2022.TICKER
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate=booktomarketdata2019Through2022.datadate
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query,conn)

crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
dividendyielddata2019Through2022.to_sql("dividendyielddata2019Through2022", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketfirstthird,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketlastthird,
       dividendyielddata2019Through2022.FirstQuartile as dividendyieldfirstquartile, dividendyielddata2019Through2022.FourthQuartile as dividendyieldfourthquartile
      FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
      LEFT JOIN dividendyielddata2019Through2022
      ON crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER=dividendyielddata2019Through2022.TICKER
      and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate=dividendyielddata2019Through2022.datadate
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query,conn)
#print(tabulate(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq, headers='keys', tablefmt='presto'))


In [ ]:
# CALCULATE YEAR AGO QUARTER SALES AND EARNINGS #

crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketfirstthird,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketlastthird,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfirstquartile,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfourthquartile
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq<>""
        ORDER BY TICKER, datadate
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query,conn)


In [ ]:
#sort the table
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.sort_values(by=['TICKER', 'datadate'], ascending=False)

#crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.drop(['saleq_4', 'ibq_4', 'fqtr_4', 'fyearq_4'])
Table2019to2022=crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019to2022['saleq_4'] = Table2019to2022.groupby(['TICKER'])['saleq'].shift(4)
Table2019to2022['ibq_4'] = Table2019to2022.groupby(['TICKER'])['ibq'].shift(4)
Table2019to2022['fqtr_4'] = Table2019to2022.groupby(['TICKER'])['fqtr'].shift(4)
Table2019to2022['fyearq_4'] = Table2019to2022.groupby(['TICKER'])['fyearq'].shift(4)
for row in Table2019to2022.index:
  try:
    if (float(Table2019to2022['fyearq'][row])-1 != float(Table2019to2022['fyearq_4'][row])) or (float(Table2019to2022['fqtr'][row]) != float(Table2019to2022['fqtr_4'][row])):
      Table2019to2022['saleq_4'][row] = ""
  except:
    continue
  try:
    if (float(Table2019to2022['fyearq'][row])-1 != float(Table2019to2022['fyearq_4'][row])) or (float(Table2019to2022['fqtr'][row]) != float(Table2019to2022['fqtr_4'][row])):
      Table2019to2022['ibq_4'][row] = ""
  except:
    continue

#sort the table
Table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq=Table2019to2022
#print(tabulate(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq, headers='keys', tablefmt='presto'))


<ipython-input-23-353b05d5fec5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2019to2022['saleq_4'][row] = ""
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-23-353b05d5fec5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2019to2022['ibq_4'][row] = ""


In [ ]:
#QUARTER AGO QUARTER SALES AND EARNINGS#
Table2019to2022 = crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
#sort the table
Table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)
print(Table2019to2022.head(10))

#crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.drop(['saleq_4', 'ibq_4', 'fqtr_4', 'fyearq_4'])

Table2019to2022['saleq_1'] = Table2019to2022.groupby(['TICKER'])['saleq'].shift(1)
Table2019to2022['ibq_1'] = Table2019to2022.groupby(['TICKER'])['ibq'].shift(1)
Table2019to2022['fqtr_1'] = Table2019to2022.groupby(['TICKER'])['fqtr'].shift(1)
Table2019to2022['fyearq_1'] = Table2019to2022.groupby(['TICKER'])['fyearq'].shift(1)

for row in Table2019to2022.index:
  try:
    if ((float(Table2019to2022['fqtr'][row])==1) and (float(Table2019to2022['fyearq'][row]-1 != float(Table2019to2022['fyearq_1'][row]))) or ((float(Table2019to2022['ftqr']['row'])==1) and (float(Table2019to2022['ftqr']['row']) != float(Table2019to2022['ftqr_1'][row])-3))):
      Table2019to2022['saleq_1'][row] = ""
  except:
    continue
  try:
    if ((float(Table2019to2022['fqtr'][row])==1) and (float(Table2019to2022['fyearq'][row]-1 != float(Table2019to2022['fyearq_1'][row]))) or ((float(Table2019to2022['ftqr']['row'])==1) and (float(Table2019to2022['ftqr']['row']) != float(Table2019to2022['ftqr_1'][row])-3))):
      Table2019to2022['ibq_1'][row] = ""
  except:
    continue
  try:
    if ((float(Table2019to2022['fqtr'][row])>1) and (float(Table2019to2022['fyearq'][row] != float(Table2019to2022['fyearq_1'][row]))) or ((float(Table2019to2022['ftqr']['row'])==1) and (float(Table2019to2022['ftqr']['row']) != float(Table2019to2022['ftqr_1'][row])+1))):
      Table2019to2022['saleq_1'][row] = ""
  except:
    continue
  try:
    if ((float(Table2019to2022['fqtr'][row])>1) and (float(Table2019to2022['fyearq'][row] != float(Table2019to2022['fyearq_1'][row]))) or ((float(Table2019to2022['ftqr']['row'])==1) and (float(Table2019to2022['ftqr']['row']) != float(Table2019to2022['ftqr_1'][row])+1))):
      Table2019to2022['ibq_1'][row] = ""
  except:
    continue

crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq=Table2019to2022
#print(tabulate(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq, headers='keys', tablefmt='presto'))


  TICKER                  rdq             datadate    dvy    saleq      ibq  \
0   None  2019-05-22 00:00:00  2019-03-31 00:00:00    0.0    0.000   -0.401   
1   None  2019-05-09 00:00:00  2019-03-31 00:00:00  140.0  902.000  167.000   
2   None  2019-05-24 00:00:00  2019-03-31 00:00:00    0.0    0.113   -0.342   
3   None  2019-08-21 00:00:00  2019-06-30 00:00:00    0.0    0.000   -0.674   
4   None  2019-08-04 00:00:00  2019-06-30 00:00:00  229.0  929.000  121.000   
5   None  2019-08-22 00:00:00  2019-06-30 00:00:00    0.0    0.265   -0.376   
6   None  2019-11-20 00:00:00  2019-09-30 00:00:00    0.0    0.000   -2.562   
7   None  2019-10-30 00:00:00  2019-09-30 00:00:00  327.0  933.000  150.000   
8   None  2019-11-25 00:00:00  2019-09-30 00:00:00    0.0    0.288   -0.375   
9   None  2020-02-26 00:00:00  2019-12-31 00:00:00    0.0    0.000   -1.465   

   fyearq  fqtr   fyr      ceqq  ...  market_value_of_equity  \
0  2019.0   1.0  12.0     1.233  ...                16.18064   
1 

<ipython-input-24-57c9b99c8248>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2019to2022['saleq_1'][row] = ""
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-24-57c9b99c8248>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2019to2022['ibq_1'][row] = ""


In [ ]:
#ONLY KEEP ROWS THAT HAVE SRW AND RW
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_4,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_4,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
       crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketfirstthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketlastthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfirstquartile,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfourthquartile,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq_4,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr_4
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_4<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_4<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_1<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_1<>""
        '''
crspcompustat2019Through2022withsaleq4andibq4 = pd.read_sql(query, conn)


In [ ]:
print(crspcompustat2019Through2022withsaleq4andibq4)

      TICKER                  rdq             datadate      dvy     saleq  \
0          A  2020-02-18 00:00:00  2020-01-31 00:00:00   56.000  1357.000   
1          A  2020-05-21 00:00:00  2020-04-30 00:00:00  111.000  1238.000   
2          A  2020-08-18 00:00:00  2020-07-31 00:00:00  167.000  1261.000   
3          A  2020-11-23 00:00:00  2020-10-31 00:00:00  222.000  1483.000   
4          A  2021-02-16 00:00:00  2021-01-31 00:00:00   59.000  1548.000   
...      ...                  ...                  ...      ...       ...   
22088   ZYXI  2021-02-25 00:00:00  2020-12-31 00:00:00    0.000    25.605   
22089   ZYXI  2021-04-29 00:00:00  2021-03-31 00:00:00    0.000    24.127   
22090   ZYXI  2021-07-29 00:00:00  2021-06-30 00:00:00    0.000    31.022   
22091   ZYXI  2021-11-02 00:00:00  2021-09-30 00:00:00    0.000    34.786   
22092   ZYXI  2022-02-24 00:00:00  2021-12-31 00:00:00    0.001    40.366   

           ibq saleq_4  ibq_4 saleq_1   ibq_1  ...  \
0      197.000  1284.

In [ ]:
# CREATE VARIABLES FOR GROWTH AND VALUE #
Table2019to2022 = crspcompustat2019Through2022withsaleq4andibq4
Table2019to2022['growthfirm'] = ""
Table2019to2022['valuefirm'] = ""
for row in Table2019to2022.index:
  try:
    if (Table2019to2022['booktomarketfirstthird'][row]) == 1 and (Table2019to2022['dividendyieldfourthquartile'][row]) != 1:
      Table2019to2022['growthfirm'][row] = True
  except:
    continue
  try:
    if (Table2019to2022['booktomarketlastthird'][row]) == 1 and (Table2019to2022['dividendyieldfourthquartile'][row]) == 1:
      Table2019to2022['valuefirm'][row] = True
  except:
    continue
crspcompustat2019Through2022withsaleq4andibq4 = Table2019to2022
#print(tabulate(Table2019to2022, headers='keys', tablefmt='presto'))
print(Table2019to2022.columns)


<ipython-input-27-a882a813b5b2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2019to2022['growthfirm'][row] = True
<ipython-input-27-a882a813b5b2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2019to2022['valuefirm'][row] = True


Index(['TICKER', 'rdq', 'datadate', 'dvy', 'saleq', 'ibq', 'saleq_4', 'ibq_4',
       'saleq_1', 'ibq_1', 'fyearq', 'fqtr', 'fyr', 'ceqq', 'cshoq', 'prccq',
       'date', 'RET', 'ret_1', 'ret_2', 'ret_3',
       'three_day_return_ifearningsduringday',
       'three_day_return_ifearningsafterhours', 'market_value_of_equity',
       'book_to_market_timet', 'booktomarketfirstthird',
       'booktomarketlastthird', 'dividendyieldfirstquartile',
       'dividendyieldfourthquartile', 'fyearq_4', 'fqtr_4', 'growthfirm',
       'valuefirm'],
      dtype='object')


In [ ]:
## CREATE VARIABLES SRW SALESSURPRISE AND EARNINGSSURPRISE #
Table2019to2022=crspcompustat2019Through2022withsaleq4andibq4
print(Table2019to2022['saleq_4'].head(10))
Table2019to2022['saleq'] = pd.to_numeric(Table2019to2022['saleq'], errors='coerce')
Table2019to2022['saleq_4'] = pd.to_numeric(Table2019to2022['saleq_4'], errors='coerce')
Table2019to2022['ibq'] = pd.to_numeric(Table2019to2022['ibq'], errors='coerce')
Table2019to2022['ibq_4'] = pd.to_numeric(Table2019to2022['ibq_4'], errors='coerce')
Table2019to2022['srwsalessurprise'] = ""
Table2019to2022['srwearningssurprise'] = ""
Table2019to2022['srwsalessurprise'] = ((Table2019to2022['saleq'])-(Table2019to2022['saleq_4']))/(Table2019to2022['saleq_4'])
Table2019to2022['srwearningssurprise'] = ((Table2019to2022['ibq'])-(Table2019to2022['ibq_4']))/(Table2019to2022['ibq_4'])
#print(tabulate(Table2019to2022, headers='keys', tablefmt='presto'))
print(Table2019to2022.columns)
crspcompustat2019Through2022withsaleq4andibq4=Table2019to2022

0    1284.0
1    1238.0
2    1274.0
3    1367.0
4    1357.0
5    1238.0
6    1261.0
7    1483.0
8    1548.0
9    2436.0
Name: saleq_4, dtype: object
Index(['TICKER', 'rdq', 'datadate', 'dvy', 'saleq', 'ibq', 'saleq_4', 'ibq_4',
       'saleq_1', 'ibq_1', 'fyearq', 'fqtr', 'fyr', 'ceqq', 'cshoq', 'prccq',
       'date', 'RET', 'ret_1', 'ret_2', 'ret_3',
       'three_day_return_ifearningsduringday',
       'three_day_return_ifearningsafterhours', 'market_value_of_equity',
       'book_to_market_timet', 'booktomarketfirstthird',
       'booktomarketlastthird', 'dividendyieldfirstquartile',
       'dividendyieldfourthquartile', 'fyearq_4', 'fqtr_4', 'growthfirm',
       'valuefirm', 'srwsalessurprise', 'srwearningssurprise'],
      dtype='object')


In [ ]:
## CREATE VARIABLES RW SALESSURPRISE AND EARNINGSSURPRISE #
Table2019to2022=crspcompustat2019Through2022withsaleq4andibq4
Table2019to2022['saleq'] = pd.to_numeric(Table2019to2022['saleq'], errors='coerce')
Table2019to2022['saleq_1'] = pd.to_numeric(Table2019to2022['saleq_1'], errors='coerce')
Table2019to2022['ibq'] = pd.to_numeric(Table2019to2022['ibq'], errors='coerce')
Table2019to2022['ibq_1'] = pd.to_numeric(Table2019to2022['ibq_1'], errors='coerce')
Table2019to2022['rwsalessurprise'] = ""
Table2019to2022['rwearningssurprise'] = ""
Table2019to2022['rwsalessurprise'] = ((Table2019to2022['saleq'])-(Table2019to2022['saleq_1']))/(Table2019to2022['saleq_1'])
Table2019to2022['rwearningssurprise'] = ((Table2019to2022['ibq'])-(Table2019to2022['ibq_1']))/(Table2019to2022['ibq_1'])
#print(tabulate(Table2019to2022, headers='keys', tablefmt='presto'))
print(Table2019to2022.columns)
crspcompustat2019Through2022withsaleq4andibq4=Table2019to2022


Index(['TICKER', 'rdq', 'datadate', 'dvy', 'saleq', 'ibq', 'saleq_4', 'ibq_4',
       'saleq_1', 'ibq_1', 'fyearq', 'fqtr', 'fyr', 'ceqq', 'cshoq', 'prccq',
       'date', 'RET', 'ret_1', 'ret_2', 'ret_3',
       'three_day_return_ifearningsduringday',
       'three_day_return_ifearningsafterhours', 'market_value_of_equity',
       'book_to_market_timet', 'booktomarketfirstthird',
       'booktomarketlastthird', 'dividendyieldfirstquartile',
       'dividendyieldfourthquartile', 'fyearq_4', 'fqtr_4', 'growthfirm',
       'valuefirm', 'srwsalessurprise', 'srwearningssurprise',
       'rwsalessurprise', 'rwearningssurprise'],
      dtype='object')


In [ ]:
### ADD IN THE IBES SURPRISE HISTORY ###
import datetime
ibessurprisehistory=pd.read_csv("/content/gdrive/MyDrive/FinancePaperFIMA/IBESSurpriseHistory.csv")
ibessurprisehistory['datadate']=datetime.datetime(2010,1,1)
ibessurprisehistory["datadate"]=pd.to_datetime(ibessurprisehistory["datadate"],format = "%Y%m%d")
ibessurprisehistory['PMON']=pd.to_numeric(ibessurprisehistory['PMON'])
ibessurprisehistory['PYEAR']=pd.to_numeric(ibessurprisehistory['PYEAR'])
ibessurprisehistory=ibessurprisehistory[ibessurprisehistory['PYEAR']>=2020]
for row in ibessurprisehistory.index:
  try:
    if ibessurprisehistory['PMON'][row]==1:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
    if ibessurprisehistory['PMON'][row]==2 and ibessurprisehistory['PYEAR'][row]==2020:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 29)
    if ibessurprisehistory['PMON'][row]==2 and ibessurprisehistory['PYEAR'][row]>2020:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 28)
    if ibessurprisehistory['PMON'][row]==3:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
    if ibessurprisehistory['PMON'][row]==4:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 30)
    if ibessurprisehistory['PMON'][row]==5:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
    if ibessurprisehistory['PMON'][row]==6:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 30)
    if ibessurprisehistory['PMON'][row]==7:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
    if ibessurprisehistory['PMON'][row]==8:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
    if ibessurprisehistory['PMON'][row]==9:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 30)
    if ibessurprisehistory['PMON'][row]==10:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
    if ibessurprisehistory['PMON'][row]==11:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 30)
    if ibessurprisehistory['PMON'][row]==12:
      ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
  except:
    continue


<ipython-input-30-2b288daab652>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 31)
<ipython-input-30-2b288daab652>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibessurprisehistory['datadate'][row]=datetime.datetime(ibessurprisehistory['PYEAR'][row],ibessurprisehistory['PMON'][row], 30)
<ipython-input-30-2b288daab652>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
## MERGE WITH THE ANNUAL AND QUARTERLY EPS AND SALES IBES SURPRISE HISTORY ##
crspcompustat2019Through2022withsaleq4andibq4.to_sql("crspcompustat2019Through2022withsaleq4andibq4",conn,if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, rdq, three_day_return_ifearningsafterhours,
        srwsalessurprise, srwearningssurprise,
        rwsalessurprise, rwearningssurprise,
        growthfirm, valuefirm,
        book_to_market_timet, market_value_of_equity
        FROM crspcompustat2019Through2022withsaleq4andibq4
        WHERE three_day_return_ifearningsafterhours<>""
        '''

crspcompustat2019Through2022allfirms = pd.read_sql(query, conn)
print(ibessurprisehistory.columns)
print(crspcompustat2019Through2022allfirms.head(10))

#ADD IN THE qtrepssuescore
ibessurprisehistory.to_sql("ibessurprisehistory",conn,if_exists='replace')
crspcompustat2019Through2022allfirms.to_sql("crspcompustat2019Through2022allfirms",conn,if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirms.TICKER, crspcompustat2019Through2022allfirms.datadate,
        crspcompustat2019Through2022allfirms.rdq, crspcompustat2019Through2022allfirms.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirms.srwsalessurprise, crspcompustat2019Through2022allfirms.srwearningssurprise,
        crspcompustat2019Through2022allfirms.rwsalessurprise, crspcompustat2019Through2022allfirms.rwearningssurprise,
        crspcompustat2019Through2022allfirms.growthfirm, crspcompustat2019Through2022allfirms.valuefirm,
        crspcompustat2019Through2022allfirms.book_to_market_timet, crspcompustat2019Through2022allfirms.market_value_of_equity,
        ibessurprisehistory.suescore as qtrepssuescore, ibessurprisehistory.anndats as qtrepsanndats
        FROM crspcompustat2019Through2022allfirms
        LEFT JOIN ibessurprisehistory
        ON crspcompustat2019Through2022allfirms.TICKER=ibessurprisehistory.TICKER and crspcompustat2019Through2022allfirms.datadate=ibessurprisehistory.datadate
        and ibessurprisehistory.FISCALP=='QTR' and ibessurprisehistory.MEASURE=="EPS"
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistory = pd.read_sql(query,conn)
### DATA FRAME BREAKS AFTER ABOVE ###
print(crspcompustat2019Through2022allfirmswithibessurprisehistory.head(10))

#ADD IN THE qtrsalessuescore
ibessurprisehistory.to_sql("ibessurprisehistory",conn,if_exists='replace')
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER, crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq, crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm, crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet, crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore, crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepsanndats,
        ibessurprisehistory.anndats as qtrsalesanndats, ibessurprisehistory.suescore as qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        LEFT JOIN ibessurprisehistory
        ON crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER=ibessurprisehistory.TICKER and
        crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate=ibessurprisehistory.datadate
        and ibessurprisehistory.FISCALP=='QTR' and ibessurprisehistory.MEASURE=="SAL"
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistory = pd.read_sql(query,conn)


Index(['TICKER', 'OFTIC', 'MEASURE', 'FISCALP', 'PYEAR', 'PMON', 'USFIRM',
       'anndats', 'actual', 'surpmean', 'surpstdev', 'suescore', 'datadate'],
      dtype='object')
  TICKER             datadate                  rdq  \
0      A  2020-01-31 00:00:00  2020-02-18 00:00:00   
1      A  2020-04-30 00:00:00  2020-05-21 00:00:00   
2      A  2020-07-31 00:00:00  2020-08-18 00:00:00   
3      A  2020-10-31 00:00:00  2020-11-23 00:00:00   
4      A  2021-01-31 00:00:00  2021-02-16 00:00:00   
5      A  2021-04-30 00:00:00  2021-05-25 00:00:00   
6      A  2021-07-31 00:00:00  2021-08-17 00:00:00   
7      A  2021-10-31 00:00:00  2021-11-22 00:00:00   
8      A  2022-01-31 00:00:00  2022-02-22 00:00:00   
9     AA  2020-12-31 00:00:00  2021-01-20 00:00:00   

   three_day_return_ifearningsafterhours  srwsalessurprise  \
0                               0.003302          0.056854   
1                               0.067246          0.000000   
2                               0.004497    

In [ ]:
print(ibessurprisehistory.columns)

Index(['TICKER', 'OFTIC', 'MEASURE', 'FISCALP', 'PYEAR', 'PMON', 'USFIRM',
       'anndats', 'actual', 'surpmean', 'surpstdev', 'suescore', 'datadate'],
      dtype='object')


In [ ]:
print(crspcompustat2019Through2022allfirms.columns)

Index(['TICKER', 'datadate', 'rdq', 'three_day_return_ifearningsafterhours',
       'srwsalessurprise', 'srwearningssurprise', 'rwsalessurprise',
       'rwearningssurprise', 'growthfirm', 'valuefirm', 'book_to_market_timet',
       'market_value_of_equity'],
      dtype='object')


In [ ]:
# CLEAN UP THE DATA FOR ALL FIRMS#
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, rdq, three_day_return_ifearningsafterhours, srwsalessurprise, srwearningssurprise, rwsalessurprise, rwearningssurprise, growthfirm,  valuefirm,  book_to_market_timet,  market_value_of_equity,  qtrepssuescore,
        qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore<>"" and crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore<>""
        '''

crspcompustat2019Through2022allfirmswithibessurprisehistory = pd.read_sql(query,conn)
print(crspcompustat2019Through2022allfirmswithibessurprisehistory)


     TICKER             datadate                  rdq  \
0        AA  2020-12-31 00:00:00  2021-01-20 00:00:00   
1        AA  2021-03-31 00:00:00  2021-04-15 00:00:00   
2        AA  2021-06-30 00:00:00  2021-07-15 00:00:00   
3        AA  2021-09-30 00:00:00  2021-10-14 00:00:00   
4        AA  2021-12-31 00:00:00  2022-01-19 00:00:00   
...     ...                  ...                  ...   
7853   ZUMZ  2022-01-31 00:00:00  2022-03-10 00:00:00   
7854    ZUO  2021-01-31 00:00:00  2021-03-11 00:00:00   
7855    ZUO  2021-07-31 00:00:00  2021-08-25 00:00:00   
7856    ZUO  2021-10-31 00:00:00  2021-12-01 00:00:00   
7857    ZUO  2022-01-31 00:00:00  2022-03-02 00:00:00   

      three_day_return_ifearningsafterhours  srwsalessurprise  \
0                                 -0.153240         -0.018062   
1                                  0.004262          0.205376   
2                                  0.011870          0.318901   
3                                  0.104527          0.

In [ ]:
# DROP INF OR N/A DATA

crspcompustat2019Through2022allfirmswithibessurprisehistory.replace([np.inf, -np.inf], np.nan, inplace=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory = crspcompustat2019Through2022allfirmswithibessurprisehistory.dropna()
#print(tabulate(crspcompustat2019Through2022allfirmswithibessurprisehistory, headers='keys', tablefmt='presto'))

In [ ]:
print

In [ ]:
## TABLE 1 PANEL FOR ALL FIRMS ##
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT TICKER
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE datadate >= "20200101"
        '''
numberofuniquefirms = pd.read_sql(query,conn)
totalnumberofuniquefirms = len(numberofuniquefirms)
print("Number of Unique Firms: "+str(len(numberofuniquefirms)))
###print out quantities next

#EARNINGS SURPRISE SRW
print("SRW Earnings Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwearningssurprise'].min()))
print("SRW Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.1)))
print("SRW Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.25)))
print("SRW Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.5)))
print("SRW Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.75)))
print("SRW Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.9)))
print("SRW Earnings Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwearningssurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwearningssurprise'])))

###SALES SURPRISE SRW
print("SRW Sales Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwsalessurprise'].min()))
print("SRW Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.1)))
print("SRW Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.25)))
print("SRW Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.5)))
print("SRW Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.75)))
print("SRW Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.9)))
print("SRW Sales Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwsalessurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwsalessurprise'])))

###EARNINGS SURPRISE RW
print("RW Earnings Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwearningssurprise'].min()))
print("RW Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.1)))
print("RW Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.25)))
print("RW Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.5)))
print("RW Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.75)))
print("RW Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.9)))
print("RW Earnings Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwearningssurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwearningssurprise'])))

###SALES SURPRISE RW
print("RW Sales Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwsalessurprise'].min()))
print("RW Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.1)))
print("RW Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.25)))
print("RW Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.5)))
print("RW Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.75)))
print("RW Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.9)))
print("RW Sales Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwsalessurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwsalessurprise'])))

###THREE DAY RETURN
print("Three Day Return Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].min()))
print("Three Day Return 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.1)))
print("Three Day Return 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.25)))
print("Three Day Return 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.5)))
print("Three Day Return 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.75)))
print("Three Day Return 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.9)))
print("Three Day Return Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'])))

###Quarterly Analyst Earnings Surprise
print("Quarterly Analyst Earnings Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescore'].min()))
print("Quarterly Analyst Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.1)))
print("Quarterly Analyst Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.25)))
print("Quarterly Analyst Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.5)))
print("Quarterly Analyst Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.75)))
print("Quarterly Analyst Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.9)))
print("Quarterly Analyst Earnings Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescore'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescore'])))

###Quarterly Analyst Sales Surprise
print("Quarterly Analyst Sales Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescore'].min()))
print("Quarterly Analyst Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.1)))
print("Quarterly Analyst Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.25)))
print("Quarterly Analyst Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.5)))
print("Quarterly Analyst Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.75)))
print("Quarterly Analyst Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.9)))
print("Quarterly Analyst Sales Surprise Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescore'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescore'])))

###BOOK TO MARKET STATISTICS
print("Book to Market Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['book_to_market_timet'].min()))
print("Book to Market 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.1)))
print("Book to Market 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.25)))
print("Book to Market 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.5)))
print("Book to Market 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.75)))
print("Book to Market 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.9)))
print("Book to Market Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['book_to_market_timet'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['book_to_market_timet'])))

###MARKET VALUE OF EQUITY
print("Market Value of Equity Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['market_value_of_equity'].min()))
print("Market Value of Equity 10th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.1)))
print("Market Value of Equity 25th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.25)))
print("Market Value of Equity 50th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.5)))
print("Market Value of Equity 75th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.75)))
print("Market Value of Equity 90th Percentile: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.9)))
print("Market Value of Equity Min: "+str(crspcompustat2019Through2022allfirmswithibessurprisehistory['market_value_of_equity'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['market_value_of_equity'])))


Number of Unique Firms: 1725
SRW Earnings Surprise Min: -1417.9177631578946
SRW Earnings Surprise 10th Percentile: -2.008298877950281
SRW Earnings Surprise 25th Percentile: -0.7167770983752308
SRW Earnings Surprise 50th Percentile: 0.04762439384479254
SRW Earnings Surprise 75th Percentile: 0.7019585542039272
SRW Earnings Surprise 90th Percentile: 2.334894596187538
SRW Earnings Surprise Min: 1412.9678456591641
Number of Observations: 7634
SRW Sales Surprise Min: -23.494949494949495
SRW Sales Surprise 10th Percentile: -0.15940847115681014
SRW Sales Surprise 25th Percentile: -0.009926356292319726
SRW Sales Surprise 50th Percentile: 0.1238435377472418
SRW Sales Surprise 75th Percentile: 0.3486309320391174
SRW Sales Surprise 90th Percentile: 0.8292342058073586
SRW Sales Surprise Min: 20469.477272727276
Number of Observations: 7634
RW Earnings Surprise Min: -1099.7152777777778
RW Earnings Surprise 10th Percentile: -1.367661214741484
RW Earnings Surprise 25th Percentile: -0.4557130432633467
R

In [ ]:
### ONLY VALUE FIRMS
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER, crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq, crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm, crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet, crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore, crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE valuefirm==1
        '''

crspcompustat2019Through2022valuefirmswithibessurprisehistory = pd.read_sql(query,conn)
print("Table 1 Summary Statistics For Value Firms")

## TABLE 1 PANEL FOR VALUE FIRMS ##
crspcompustat2019Through2022valuefirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022valuefirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT TICKER
        FROM crspcompustat2019Through2022valuefirmswithibessurprisehistory
        '''
numberofuniquefirms = pd.read_sql(query,conn)
totalnumberofuniquefirms = len(numberofuniquefirms)
print("Number of Unique Firms: "+str(len(numberofuniquefirms)))
###print out quantities next

#EARNINGS SURPRISE SRW
print("SRW Earnings Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwearningssurprise'].min()))
print("SRW Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.1)))
print("SRW Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.25)))
print("SRW Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.5)))
print("SRW Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.75)))
print("SRW Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.9)))
print("SRW Earnings Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwearningssurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwearningssurprise'])))


###SALES SURPRISE SRW
print("SRW Sales Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwsalessurprise'].min()))
print("SRW Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.1)))
print("SRW Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.25)))
print("SRW Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.5)))
print("SRW Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.75)))
print("SRW Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.9)))
print("SRW Sales Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwsalessurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwsalessurprise'])))


###EARNINGS SURPRISE RW
print("RW Earnings Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwearningssurprise'].min()))
print("RW Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.1)))
print("RW Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.25)))
print("RW Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.5)))
print("RW Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.75)))
print("RW Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.9)))
print("RW Earnings Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwearningssurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwearningssurprise'])))


###SALES SURPRISE RW
print("RW Sales Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwsalessurprise'].min()))
print("RW Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.1)))
print("RW Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.25)))
print("RW Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.5)))
print("RW Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.75)))
print("RW Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.9)))
print("RW Sales Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwsalessurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwsalessurprise'])))


###THREE DAY RETURN
print("Three Day Return Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].min()))
print("Three Day Return 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.1)))
print("Three Day Return 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.25)))
print("Three Day Return 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.5)))
print("Three Day Return 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.75)))
print("Three Day Return 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.9)))
print("Three Day Return Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['three_day_return_ifearningsafterhours'])))
###Quarterly Analyst Earnings Surprise
print("Quarterly Analyst Earnings Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrepssuescore'].min()))
print("Quarterly Analyst Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.1)))
print("Quarterly Analyst Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.25)))
print("Quarterly Analyst Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.5)))
print("Quarterly Analyst Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.75)))
print("Quarterly Analyst Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.9)))
print("Quarterly Analyst Earnings Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrepssuescore'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrepssuescore'])))
###Quarterly Analyst Sales Surprise
print("Quarterly Analyst Sales Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrsalessuescore'].min()))
print("Quarterly Analyst Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.1)))
print("Quarterly Analyst Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.25)))
print("Quarterly Analyst Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.5)))
print("Quarterly Analyst Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.75)))
print("Quarterly Analyst Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.9)))
print("Quarterly Analyst Sales Surprise Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrsalessuescore'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrsalessuescore'])))
###BOOK TO MARKET STATISTICS
print("Book to Market Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['book_to_market_timet'].min()))
print("Book to Market 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.1)))
print("Book to Market 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.25)))
print("Book to Market 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.5)))
print("Book to Market 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.75)))
print("Book to Market 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.9)))
print("Book to Market Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['book_to_market_timet'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['book_to_market_timet'])))
###MARKET VALUE OF EQUITY
print("Market Value of Equity Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['market_value_of_equity'].min()))
print("Market Value of Equity 10th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.1)))
print("Market Value of Equity 25th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.25)))
print("Market Value of Equity 50th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.5)))
print("Market Value of Equity 75th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.75)))
print("Market Value of Equity 90th Percentile: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.9)))
print("Market Value of Equity Min: "+str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['market_value_of_equity'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['market_value_of_equity'])))


Table 1 Summary Statistics For Value Firms
Number of Unique Firms: 295
SRW Earnings Surprise Min: -333.12162162162167
SRW Earnings Surprise 10th Percentile: -1.8665298904765661
SRW Earnings Surprise 25th Percentile: -0.6884312584436425
SRW Earnings Surprise 50th Percentile: 0.06505316085206014
SRW Earnings Surprise 75th Percentile: 0.644750360876679
SRW Earnings Surprise 90th Percentile: 1.776059617765947
SRW Earnings Surprise Min: 86.75550660792952
Number of Observations: 1056
SRW Sales Surprise Min: -23.494949494949495
SRW Sales Surprise 10th Percentile: -0.21448924997347685
SRW Sales Surprise 25th Percentile: -0.0920638566801485
SRW Sales Surprise 50th Percentile: 0.0021599384654677027
SRW Sales Surprise 75th Percentile: 0.1246284233265425
SRW Sales Surprise 90th Percentile: 0.37351443315920485
SRW Sales Surprise Min: 14.849274481501176
Number of Observations: 1056
RW Earnings Surprise Min: -74.13207547169812
RW Earnings Surprise 10th Percentile: -1.2992922748991103
RW Earnings Surp

In [ ]:
print(crspcompustat2019Through2022valuefirmswithibessurprisehistory)

     TICKER             datadate                  rdq  \
0      AAIC  2021-09-30 00:00:00  2021-11-09 00:00:00   
1      AAIC  2021-12-31 00:00:00  2022-03-02 00:00:00   
2       ABM  2020-10-31 00:00:00  2020-12-16 00:00:00   
3       ABM  2021-04-30 00:00:00  2021-06-08 00:00:00   
4       ABM  2021-07-31 00:00:00  2021-09-08 00:00:00   
...     ...                  ...                  ...   
1051   ZION  2020-12-31 00:00:00  2021-01-19 00:00:00   
1052   ZION  2021-03-31 00:00:00  2021-04-19 00:00:00   
1053   ZION  2021-06-30 00:00:00  2021-07-19 00:00:00   
1054   ZION  2021-09-30 00:00:00  2021-10-18 00:00:00   
1055   ZION  2021-12-31 00:00:00  2022-01-24 00:00:00   

      three_day_return_ifearningsafterhours  srwsalessurprise  \
0                                 -0.007792         -0.462401   
1                                  0.072072         -0.429642   
2                                 -0.040655         -0.099150   
3                                 -0.009401          0.

In [ ]:
### TABLE 1 FOR ONLY GROWTH FIRMS

crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT
        crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE growthfirm==1
        '''

crspcompustat2019Through2022growthfirmswithibessurprisehistory = pd.read_sql(query,conn)
print("Table 1 Summary Statistics For Growth Firms")

## TABLE 1 PANEL FOR GROWTH FIRMS ##
crspcompustat2019Through2022growthfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022growthfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT TICKER
        FROM crspcompustat2019Through2022growthfirmswithibessurprisehistory
        WHERE datadate >= "20200101"
        '''
numberofuniquefirms = pd.read_sql(query,conn)
totalnumberofuniquefirms = len(numberofuniquefirms)
print("Number of Unique Firms: "+str(len(numberofuniquefirms)))
###print out quantities next

#EARNINGS SURPRISE SRW
print("SRW Earnings Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwearningssurprise'].min()))
print("SRW Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.1)))
print("SRW Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.25)))
print("SRW Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.5)))
print("SRW Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.75)))
print("SRW Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.9)))
print("SRW Earnings Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwearningssurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwearningssurprise'])))


###SALES SURPRISE SRW
print("SRW Sales Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwsalessurprise'].min()))
print("SRW Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.1)))
print("SRW Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.25)))
print("SRW Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.5)))
print("SRW Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.75)))
print("SRW Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.9)))
print("SRW Sales Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwsalessurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwsalessurprise'])))


###EARNINGS SURPRISE RW
print("RW Earnings Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwearningssurprise'].min()))
print("RW Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.1)))
print("RW Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.25)))
print("RW Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.5)))
print("RW Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.75)))
print("RW Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.9)))
print("RW Earnings Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwearningssurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwearningssurprise'])))


###SALES SURPRISE RW
print("RW Sales Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwsalessurprise'].min()))
print("RW Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.1)))
print("RW Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.25)))
print("RW Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.5)))
print("RW Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.75)))
print("RW Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.9)))
print("RW Sales Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwsalessurprise'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwsalessurprise'])))


###THREE DAY RETURN
print("Three Day Return Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].min()))
print("Three Day Return 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.1)))
print("Three Day Return 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.25)))
print("Three Day Return 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.5)))
print("Three Day Return 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.75)))
print("Three Day Return 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.9)))
print("Three Day Return Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'])))
###Quarterly Analyst Earnings Surprise
print("Quarterly Analyst Earnings Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrepssuescore'].min()))
print("Quarterly Analyst Earnings Surprise 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.1)))
print("Quarterly Analyst Earnings Surprise 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.25)))
print("Quarterly Analyst Earnings Surprise 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.5)))
print("Quarterly Analyst Earnings Surprise 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.75)))
print("Quarterly Analyst Earnings Surprise 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.9)))
print("Quarterly Analyst Earnings Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrepssuescore'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrepssuescore'])))
###Quarterly Analyst Sales Surprise
print("Quarterly Analyst Sales Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrsalessuescore'].min()))
print("Quarterly Analyst Sales Surprise 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.1)))
print("Quarterly Analyst Sales Surprise 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.25)))
print("Quarterly Analyst Sales Surprise 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.5)))
print("Quarterly Analyst Sales Surprise 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.75)))
print("Quarterly Analyst Sales Surprise 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.9)))
print("Quarterly Analyst Sales Surprise Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrsalessuescore'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrsalessuescore'])))
###BOOK TO MARKET STATISTICS
print("Book to Market Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['book_to_market_timet'].min()))
print("Book to Market 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.1)))
print("Book to Market 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.25)))
print("Book to Market 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.5)))
print("Book to Market 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.75)))
print("Book to Market 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.9)))
print("Book to Market Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['book_to_market_timet'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['book_to_market_timet'])))
###MARKET VALUE OF EQUITY
print("Market Value of Equity Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['market_value_of_equity'].min()))
print("Market Value of Equity 10th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.1)))
print("Market Value of Equity 25th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.25)))
print("Market Value of Equity 50th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.5)))
print("Market Value of Equity 75th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.75)))
print("Market Value of Equity 90th Percentile: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.9)))
print("Market Value of Equity Min: "+str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['market_value_of_equity'].max()))
print("Number of Observations: "+str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['market_value_of_equity'])))


Table 1 Summary Statistics For Growth Firms
Number of Unique Firms: 715
SRW Earnings Surprise Min: -824.4615384615385
SRW Earnings Surprise 10th Percentile: -1.7226242676012338
SRW Earnings Surprise 25th Percentile: -0.447170042818798
SRW Earnings Surprise 50th Percentile: 0.14376817333030267
SRW Earnings Surprise 75th Percentile: 0.8126027132169014
SRW Earnings Surprise 90th Percentile: 2.2473679471593733
SRW Earnings Surprise Min: 736.4399999999999
Number of Observations: 2532
SRW Sales Surprise Min: -5.813664596273292
SRW Sales Surprise 10th Percentile: -0.07617006642566994
SRW Sales Surprise 25th Percentile: 0.06931867870726435
SRW Sales Surprise 50th Percentile: 0.21856331943718244
SRW Sales Surprise 75th Percentile: 0.5018142124203795
SRW Sales Surprise 90th Percentile: 1.095929480429616
SRW Sales Surprise Min: 20469.477272727276
Number of Observations: 2532
RW Earnings Surprise Min: -1099.7152777777778
RW Earnings Surprise 10th Percentile: -1.2038700981051966
RW Earnings Surpris

In [ ]:
print(crspcompustat2019Through2022growthfirmswithibessurprisehistory)

     TICKER             datadate                  rdq  \
0      AAON  2020-12-31 00:00:00  2021-02-25 00:00:00   
1      AAON  2021-03-31 00:00:00  2021-05-06 00:00:00   
2      AAON  2021-06-30 00:00:00  2021-08-05 00:00:00   
3      AAON  2021-09-30 00:00:00  2021-11-04 00:00:00   
4      AAON  2021-12-31 00:00:00  2022-02-28 00:00:00   
...     ...                  ...                  ...   
2527     ZS  2022-01-31 00:00:00  2022-02-24 00:00:00   
2528    ZUO  2021-01-31 00:00:00  2021-03-11 00:00:00   
2529    ZUO  2021-07-31 00:00:00  2021-08-25 00:00:00   
2530    ZUO  2021-10-31 00:00:00  2021-12-01 00:00:00   
2531    ZUO  2022-01-31 00:00:00  2022-03-02 00:00:00   

      three_day_return_ifearningsafterhours  srwsalessurprise  \
0                                 -0.022315         -0.047922   
1                                  0.019056         -0.157801   
2                                  0.109442          0.145546   
3                                  0.036506          0.

In [ ]:
### WINZORIZE THE QUARTERLY EPS SURPRISE ###
### WINZORIZE qtrepssuescore ###
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'] = crspcompustat2019Through2022allfirmswithibessurprisehistory.groupby(['datadate'])['qtrepssuescore'].rank(pct=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorefirstpercentile']=""
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescoreninetyninthpercentile']=""
for row in crspcompustat2019Through2022allfirmswithibessurprisehistory.index:
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row]<=.01:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorefirstpercentile'][row] = 1
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row]>=.99:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescoreninetyninthpercentile'][row] = 1
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row]<=.01 and crspcompustat2019Through2022allfirmswithibessurprisehistory['percentile'][row]<.99:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorefirstpercentile'][row] = 0
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row]<=.01 and crspcompustat2019Through2022allfirmswithibessurprisehistory['percentile'][row]<.99:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescoreninetyninthpercentile'][row] = 0
  except:
    continue

#WINSORIZE QUARTERLY SALES SURPRISE#
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'] = crspcompustat2019Through2022allfirmswithibessurprisehistory.groupby(['datadate'])['qtrsalessuescore'].rank(pct=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorefirstpercentile']=""
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescoreninetyninthpercentile']=""
for row in crspcompustat2019Through2022allfirmswithibessurprisehistory.index:
 try:
   if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row]<=.01:
     crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorefirstpercentile'][row] = 1
 except:
   continue
 try:
   if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row]>=.99:
     crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescoreninetyninthpercentile'][row] = 1
 except:
   continue
 try:
   if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row]<=.01 and crspcompustat2019Through2022allfirmswithibessurprisehistory['percentile'][row]<.99:
     crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorefirstpercentile'][row] = 0
 except:
   continue
 try:
   if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row]<=.01 and crspcompustat2019Through2022allfirmswithibessurprisehistory['percentile'][row]<.99:
     crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescoreninetyninthpercentile'][row] = 0
 except:
   continue

### WINZORIZE THREE DAY RETURN ###
crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnpercentile'] = crspcompustat2019Through2022allfirmswithibessurprisehistory.groupby(['datadate'])['three_day_return_ifearningsafterhours'].rank(pct=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnfirstpercentile']=""
crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnninetyninthpercentile']=""
for row in crspcompustat2019Through2022allfirmswithibessurprisehistory.index:
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnpercentile'][row]<=.01:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnfirstpercentile'][row] = 1
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnpercentile'][row]>=.99:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnninetyninthpercentile'][row] = 1
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnpercentile'][row]<=.01 and crspcompustat2019Through2022allfirmswithibessurprisehistory['percentile'][row]<.99:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnfirstpercentile'][row] = 0
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnpercentile'][row]<=.01 and crspcompustat2019Through2022allfirmswithibessurprisehistory['percentile'][row]<.99:
      crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnninetyninthpercentile'][row] = 0
  except:
    continue

crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory",conn,if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, rdq, three_day_return_ifearningsafterhours, srwsalessurprise,
        srwearningssurprise, rwsalessurprise, rwearningssurprise,
        qtrepssuescore, qtrsalessuescore, growthfirm, valuefirm, market_value_of_equity,
        qtrepssuescorefirstpercentile, qtrepssuescoreninetyninthpercentile, qtrsalessuescorefirstpercentile, qtrsalessuescoreninetyninthpercentile,
        threedayreturnfirstpercentile, threedayreturnninetyninthpercentile
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE three_day_return_ifearningsafterhours<>"" and qtrepssuescore<>"" and qtrsalessuescore<>"" and qtrepssuescorefirstpercentile<>1
        and qtrepssuescoreninetyninthpercentile<>1 and qtrsalessuescorefirstpercentile<>1 and qtrsalessuescoreninetyninthpercentile<>1
        and threedayreturnfirstpercentile<>1 and threedayreturnninetyninthpercentile<>1
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore = pd.read_sql(query,conn)
print(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.head(10))


<ipython-input-41-507ac0eff614>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorefirstpercentile'][row] = 1
<ipython-input-41-507ac0eff614>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescoreninetyninthpercentile'][row] = 1
<ipython-input-41-507ac0eff614>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

  TICKER             datadate                  rdq  \
0     AA  2020-12-31 00:00:00  2021-01-20 00:00:00   
1     AA  2021-03-31 00:00:00  2021-04-15 00:00:00   
2     AA  2021-06-30 00:00:00  2021-07-15 00:00:00   
3     AA  2021-09-30 00:00:00  2021-10-14 00:00:00   
4     AA  2021-12-31 00:00:00  2022-01-19 00:00:00   
5   AAIC  2021-09-30 00:00:00  2021-11-09 00:00:00   
6   AAIC  2021-12-31 00:00:00  2022-03-02 00:00:00   
7    AAN  2021-09-30 00:00:00  2021-10-26 00:00:00   
8    AAN  2021-12-31 00:00:00  2022-02-23 00:00:00   
9   AAOI  2021-03-31 00:00:00  2021-05-06 00:00:00   

   three_day_return_ifearningsafterhours  srwsalessurprise  \
0                              -0.153240         -0.018062   
1                               0.004262          0.205376   
2                               0.011870          0.318901   
3                               0.104527          0.314588   
4                              -0.026999          0.396321   
5                              -0

In [ ]:
# RUN REGRESSION OF THREE DAY RETURN #
import statsmodels.formula.api as sm
from scipy.stats.mstats import winsorize
import pandas
import statsmodels.api as sm

## GET RID OF INF VALUES ##
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.replace([np.inf, -np.inf], np.nan, inplace=True)
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.dropna()
print(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.isna().sum())
print(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.head(10))
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.to_csv('/content/gdrive/MyDrive/FinancePaperFIMA/crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.csv')

# KEEP ONLY THE 2020 DATA #
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore",conn,if_exists='replace')
query = '''
        SELECT DISTINCT *
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore
        WHERE datadate>="2020-01-01"
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore = pd.read_sql(query,conn)

##REPRODUCE TABLE 2##
threedayreturnmodeltable2 = ols("three_day_return_ifearningsafterhours ~ qtrsalessuescore + qtrepssuescore", data=crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()

#summarize our model
threedayreturnmodeltable2_summary = threedayreturnmodeltable2.summary()
print(threedayreturnmodeltable2_summary)


TICKER                                   0
datadate                                 0
rdq                                      0
three_day_return_ifearningsafterhours    0
srwsalessurprise                         0
srwearningssurprise                      0
rwsalessurprise                          0
rwearningssurprise                       0
qtrepssuescore                           0
qtrsalessuescore                         0
growthfirm                               0
valuefirm                                0
market_value_of_equity                   0
qtrepssuescorefirstpercentile            0
qtrepssuescoreninetyninthpercentile      0
qtrsalessuescorefirstpercentile          0
qtrsalessuescoreninetyninthpercentile    0
threedayreturnfirstpercentile            0
threedayreturnninetyninthpercentile      0
dtype: int64
  TICKER             datadate                  rdq  \
0     AA  2020-12-31 00:00:00  2021-01-20 00:00:00   
1     AA  2021-03-31 00:00:00  2021-04-15 00:00:00   
2     AA

In [ ]:
### RUN A REGRESSION FOR EVERYTHING WITH DUMMY ###
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']=""
for row in crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.index:
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuefirm'][row]=="1":
      crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'][row]=1
  except:
    continue
  try:
    if crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuefirm'][row]=="":
      crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'][row]=0
  except:
    continue

crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore']=pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore'])
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']=pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'])
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore']=pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore'])
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescoreXvaluedummy']=crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore']*crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescoreXvaluedummy']=crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore']*crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']
print(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.head(10))

### REPRODUCE TABLE 3 ###
threedayreturnmodeltable3 = ols("three_day_return_ifearningsafterhours ~ valuedummy + qtrsalessuescore + qtrepssuescore + qtrsalessuescoreXvaluedummy + qtrepssuescoreXvaluedummy", data=crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()

# summarize our model
threedayreturnmodeltable3_summary = threedayreturnmodeltable3.summary()
print(threedayreturnmodeltable3_summary)

#result = sns.ols(formula="three_day_return_ifearningsafterhours ~ earningssurprise + salessurprise", data=crspcompustat2019Through2022growthfirms).fit()
#print(result.params)

<ipython-input-43-6fb20989421b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'][row]=0
<ipython-input-43-6fb20989421b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'][row]=1


   index TICKER             datadate                  rdq  \
0      0     AA  2020-12-31 00:00:00  2021-01-20 00:00:00   
1      1     AA  2021-03-31 00:00:00  2021-04-15 00:00:00   
2      2     AA  2021-06-30 00:00:00  2021-07-15 00:00:00   
3      3     AA  2021-09-30 00:00:00  2021-10-14 00:00:00   
4      4     AA  2021-12-31 00:00:00  2022-01-19 00:00:00   
5      5   AAIC  2021-09-30 00:00:00  2021-11-09 00:00:00   
6      6   AAIC  2021-12-31 00:00:00  2022-03-02 00:00:00   
7      7    AAN  2021-09-30 00:00:00  2021-10-26 00:00:00   
8      8    AAN  2021-12-31 00:00:00  2022-02-23 00:00:00   
9      9   AAOI  2021-03-31 00:00:00  2021-05-06 00:00:00   

   three_day_return_ifearningsafterhours  srwsalessurprise  \
0                              -0.153240         -0.018062   
1                               0.004262          0.205376   
2                               0.011870          0.318901   
3                               0.104527          0.314588   
4                 